# Air Quality 資料清理
### Tasks
* 1.刪除全NA列、行，fill NA值
* 2.統一、轉換欄位命名 (EX: "序號"->"number")
* 3.確認data type 
* 4.merge duplicate data
* 5.split date column to "year","month","day"
* 6.刪除column、reset index
* 7.insert to mongodb

In [64]:
import pandas as pd
import json
import pymongo
import os

# file path
AQ_folder_path = "day_value"
key_name_path_EN = 'changeKeyName.json'
value_name_path_EN = 'day_value_CH2EN.json'


# read file
key_name_EN = pd.read_json(key_name_path_EN, typ='series')
value_name_EN = pd.read_json(value_name_path_EN, typ='series')
week_day_key = {'0':'Mon','1':'Tue','2':'Wed','3':'Thu','4':'Fri','5':'Sat','6':'Sun'}

# log in db
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["dataset"]

is_EN = False
mycol = mydb["AQ_EN"] if is_EN else mydb["AQ_CH"]
#mycol.delete_many({})

In [65]:
def clean_data(data,is_EN):
    # 1.刪除全NA列、行
    data.dropna(axis=0,how="all",inplace=True)
    data.dropna(axis=1,how="all",inplace=True)

    # 填充缺值: replace '-' to 0
    data = data.replace('-',0)
    
    # 2.改column名稱、改value 名稱
    data.rename(columns=key_name_EN,inplace=True)
    if is_EN:
        data.replace(value_name_EN,inplace=True)
    
    # 3. 確認資料型別
    data = data.astype({ "AMB_TEMP(C)": "float",
        "CO(ppm)": "float",
        #"CO2(ppm)": "float",
        "NO(ppb)": "float",
        "NO2(ppb)": "float",
        "NOx(ppb)": "float",
        "O3(ppb)": "float",
        "PM10(ug/m3)": "float",
        "PM2_5(ug/m3)": "float",
        "RH(percent)": "float",
        "SO2(ppb)": "float",
        "WIND_SPEED(m/sec)": "float",
        "city": "category",
        "station": "category",
               })
    
    # 4.merge duplicate rows
    data=data.groupby(['city','station','date']).sum().reset_index()
    indexNames = data[data['PM2_5(ug/m3)'] == 0.0 ].index
    data.drop(indexNames , inplace=True)
    
    # 5. 時間資訊切成年、月、日
    data[["year","month","day"]] = data['date'].str.split('/',expand=True)

    # 新增星期幾
    data["weekDay"] = pd.to_datetime(data["date"]).dt.dayofweek.astype(int).astype(str).map(week_day_key)

    # 6.刪除欄位
    data.drop(columns="date",inplace=True)
    data.drop(columns="number",inplace=True)
    data.reset_index(drop=True,inplace=True)

    # reset_index
    data.reset_index(inplace=True)
    data.rename(columns={"index":"number"},inplace=True)

    # 7. insert to mongo db
    parsed_js = json.loads(data.to_json(orient = 'records',force_ascii=False))    
    mycol.insert_many(parsed_js)
    
    print(data.shape)
    print()

In [66]:
# 清理2014-2018資料
for folder in os.listdir(AQ_folder_path):
    for fileName in os.listdir(AQ_folder_path+'\\'+folder+'\\csv'):
        
        csv_fpath = AQ_folder_path+'\\'+folder+'\\csv\\'+fileName
        print(csv_fpath)
        data = pd.read_csv(csv_fpath)
        clean_data(data,is_EN)

day_value\2014\csv\空氣品質監測日值2014-01.csv
(2331, 18)

day_value\2014\csv\空氣品質監測日值2014-02.csv
(2115, 18)

day_value\2014\csv\空氣品質監測日值2014-03.csv
(2343, 18)

day_value\2014\csv\空氣品質監測日值2014-04.csv
(2264, 18)

day_value\2014\csv\空氣品質監測日值2014-05.csv
(2309, 18)

day_value\2014\csv\空氣品質監測日值2014-06.csv
(2240, 18)

day_value\2014\csv\空氣品質監測日值2014-07.csv
(2319, 18)

day_value\2014\csv\空氣品質監測日值2014-08.csv
(2309, 18)

day_value\2014\csv\空氣品質監測日值2014-09.csv
(2258, 18)

day_value\2014\csv\空氣品質監測日值2014-10.csv
(2345, 18)

day_value\2014\csv\空氣品質監測日值2014-11.csv
(2265, 18)

day_value\2014\csv\空氣品質監測日值2014-12.csv
(2329, 18)

day_value\2015\csv\空氣品質監測日值2015-01.csv
(2338, 18)

day_value\2015\csv\空氣品質監測日值2015-02.csv
(2114, 18)

day_value\2015\csv\空氣品質監測日值2015-03.csv
(2301, 18)

day_value\2015\csv\空氣品質監測日值2015-04.csv
(2211, 18)

day_value\2015\csv\空氣品質監測日值2015-05.csv
(2312, 18)

day_value\2015\csv\空氣品質監測日值2015-06.csv
(2252, 18)

day_value\2015\csv\空氣品質監測日值2015-07.csv
(2324, 18)

day_value\2015\csv\空氣品質監測日值2015